In [ ]:
!!pip install -q git+https://github.com/keras-team/keras-hub.git
!!pip install -q --upgrade keras  # Upgrade to Keras 3.

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # @param ["tensorflow", "jax", "torch"]

import json
import math
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras import losses
from keras import ops
from keras import optimizers
from keras.optimizers import schedules
from keras import metrics
from keras.applications.imagenet_utils import decode_predictions
import keras_hub

# Import tensorflow for `tf.data` and its preprocessing functions
import tensorflow as tf
import tensorflow_datasets as tfds


In [ ]:
classifier = keras_hub.models.ImageClassifier.from_preset("resnet_v2_50_imagenet")

In [ ]:
def lr_warmup_cosine_decay(
    global_step,
    warmup_steps,
    hold=0,
    total_steps=0,
    start_lr=0.0,
    target_lr=1e-2,
):
    # Cosine decay
    learning_rate = (
        0.5
        * target_lr
        * (
            1
            + ops.cos(
                math.pi
                * ops.convert_to_tensor(
                    global_step - warmup_steps - hold, dtype="float32"
                )
                / ops.convert_to_tensor(
                    total_steps - warmup_steps - hold, dtype="float32"
                )
            )
        )
    )

    warmup_lr = target_lr * (global_step / warmup_steps)

    if hold > 0:
        learning_rate = ops.where(
            global_step > warmup_steps + hold, learning_rate, target_lr
        )

    learning_rate = ops.where(global_step < warmup_steps, warmup_lr, learning_rate)
    return learning_rate


class WarmUpCosineDecay(schedules.LearningRateSchedule):
    def __init__(self, warmup_steps, total_steps, hold, start_lr=0.0, target_lr=1e-2):
        super().__init__()
        self.start_lr = start_lr
        self.target_lr = target_lr
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.hold = hold

    def __call__(self, step):
        lr = lr_warmup_cosine_decay(
            global_step=step,
            total_steps=self.total_steps,
            warmup_steps=self.warmup_steps,
            start_lr=self.start_lr,
            target_lr=self.target_lr,
            hold=self.hold,
        )
        return ops.where(step > self.total_steps, 0.0, lr)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Parameters
BATCH_SIZE = 32
IMAGE_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
NUM_CLASSES = 4
NUM_FOLDS = 5
EPOCHS = 30
GAMMA = 1.5


In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/praeciso/tt",
    image_size=IMAGE_SIZE,
    batch_size=1,
    label_mode='int'
)

images = []
labels = []
for image, label in dataset:
    images.append(image[0])
    labels.append(label[0])


images = np.array([img.numpy() for img in images])
labels = np.array([lbl.numpy() for lbl in labels])

# Check the shape to confirm
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt

def augment_dataset(images, labels, gamma_value):
    gamma_images = tf.image.adjust_gamma(images, gamma=gamma_value).numpy()
    augmented_images = np.concatenate((images, gamma_images), axis=0)
    augmented_labels = np.concatenate((labels, labels), axis=0)
    return augmented_images, augmented_labels

# Define function to visualize gamma augmentation
def visualize_gamma_augmentation(images, gamma_value):
    gamma_images = tf.image.adjust_gamma(images, gamma=gamma_value).numpy()

    # Display original and gamma-corrected images side-by-side
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    for i in range(5):
        axes[0, i].imshow(images[i].astype("uint8"))
        axes[0, i].axis("off")
        axes[0, i].set_title("Original")

        axes[1, i].imshow(gamma_images[i].astype("uint8"))
        axes[1, i].axis("off")
        axes[1, i].set_title(f"Gamma: {gamma_value}")

    plt.suptitle("Gamma Augmentation Visualization")
    plt.tight_layout()
    plt.show()

# Apply gamma augmentation and print dataset sizes
def augment_and_check(images, labels, gamma_value):
    print(f"Original dataset size: {len(images)} images")
    augmented_images, augmented_labels = augment_dataset(images, labels, gamma_value)
    print(f"Dataset size after augmentation: {len(augmented_images)} images")

    # Visualize augmentation
    visualize_gamma_augmentation(images, gamma_value)
    return augmented_images, augmented_labels

# Example: Validate gamma augmentation with gamma=1.5
gamma_value_to_check = 1.5
augmented_images, augmented_labels = augment_and_check(images[:10], labels[:10], gamma_value_to_check)  # Use a subset for visualization


In [ ]:
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Hyperparameters and initialization
gamma_values = [1.2, 1.4, 1.5]  # Gamma values to test
batch_size = 64
BATCH_SIZE = 64
EPOCHS = 15
NUM_CLASSES = 4
AUTOTUNE = tf.data.AUTOTUNE
IMAGE_SIZE = (224, 224)  # Define image size for resizing

# Nested CV setup
outer_cv = KFold(n_splits=3, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Store results
outer_results = []
validation_histories = []  # Track validation accuracy for each gamma

# Define function for dataset augmentation
def augment_dataset(images, labels, gamma_value):
    gamma_images = tf.image.adjust_gamma(images, gamma=gamma_value).numpy()
    augmented_images = np.concatenate((images, gamma_images), axis=0)
    augmented_labels = np.concatenate((labels, labels), axis=0)
    return augmented_images, augmented_labels

# Load and preprocess dataset
def load_dataset(directory, image_size):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        image_size=image_size,
        label_mode="int",
        batch_size=None,  # No batching here, we'll handle batching later
    )
    images = []
    labels = []
    for image, label in dataset:
        images.append(image)
        labels.append(label)
    images = np.stack(images)
    labels = np.array(labels)
    return images, labels

# Read dataset
images, labels = load_dataset("/content/drive/MyDrive/praeciso/tt", IMAGE_SIZE)
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

# Outer CV loop
for outer_fold, (train_val_idx, test_idx) in enumerate(outer_cv.split(images)):
    gamma_value = gamma_values[outer_fold % len(gamma_values)]  # Assign gamma value for this fold
    print(f"Outer Fold {outer_fold + 1}: Using Gamma Value = {gamma_value}")

    # Split data into train+val and test sets
    X_train_val, X_test = images[train_val_idx], images[test_idx]
    y_train_val, y_test = labels[train_val_idx], labels[test_idx]

    # Augment the training+validation dataset with the selected gamma value
    aug_train_val_images, aug_train_val_labels = augment_dataset(X_train_val, y_train_val, gamma_value)

    best_inner_model = None
    best_inner_accuracy = 0
    fold_validation_accuracies = []

    # Inner CV loop
    for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(aug_train_val_images)):
        # Split into train and validation sets
        X_train, X_val = aug_train_val_images[train_idx], aug_train_val_images[val_idx]
        y_train, y_val = aug_train_val_labels[train_idx], aug_train_val_labels[val_idx]

        # Create TF datasets
        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        train_dataset = train_dataset.batch(batch_size).prefetch(AUTOTUNE)

        val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
        val_dataset = val_dataset.batch(batch_size).prefetch(AUTOTUNE)

        # Define and compile the model
        base_model = keras_hub.models.ImageClassifier.from_preset("resnet_v2_50_imagenet", num_classes=4, trainable=True)
        inputs = tf.keras.Input(shape=(224, 224, 3))
        x = base_model(inputs)
        outputs = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")(x)
        model = tf.keras.Model(inputs, outputs)

        print("Length of augmented training + validation images:", len(aug_train_val_images))
        print("Length of augmented training + validation labels:", len(aug_train_val_labels))

        total_images = len(aug_train_val_images)
        total_steps = (total_images // BATCH_SIZE) * EPOCHS
        warmup_steps = int(0.1 * total_steps)
        hold_steps = int(0.45 * total_steps)
        schedule = WarmUpCosineDecay(
            start_lr=0.05,
            target_lr=1e-2,
            warmup_steps=warmup_steps,
            total_steps=total_steps,
            hold=hold_steps,
        )
        model.compile(
            loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.SGD(weight_decay=5e-4, learning_rate=schedule, momentum=0.9,),
            metrics=["accuracy"],
        )

        # Train the model
        history = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset, verbose=1)

        # Store validation accuracy for plotting
        fold_validation_accuracies.append(history.history["val_accuracy"])

        # Evaluate validation accuracy
        val_accuracy = history.history["val_accuracy"][-1]
        print(f"Inner Fold {inner_fold + 1}: Validation Accuracy = {val_accuracy}")

        # Update best model for inner loop
        if val_accuracy > best_inner_accuracy:
            best_inner_accuracy = val_accuracy
            best_inner_model = model.get_weights()

    # Track validation history for visualization
    validation_histories.append(fold_validation_accuracies)

    # Test the best model from inner CV on the outer test set
    best_model = tf.keras.Model(inputs=inputs, outputs=outputs)
    best_model.set_weights(best_inner_model)

    # Recompile the model after loading weights
    best_model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
        metrics=["accuracy"],
    )

    # Evaluate on the test set
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    test_dataset = test_dataset.batch(batch_size).prefetch(AUTOTUNE)
    test_accuracy = best_model.evaluate(test_dataset, verbose=0)[1]
    print(f"Outer Fold {outer_fold + 1}: Test Accuracy = {test_accuracy}")
    outer_results.append(test_accuracy)

    # Plot validation accuracy progression
    plt.figure(figsize=(10, 5))
    for idx, gamma_acc in enumerate(fold_validation_accuracies):
        plt.plot(gamma_acc, label=f"Inner Fold {idx + 1}")
    plt.title(f"Outer Fold {outer_fold + 1}: Validation Accuracy Progression")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

# Final results
print(f"Nested CV Average Accuracy: {np.mean(outer_results)}")
